INGESTÃO DE DADOS


In [3]:
import pandas as pd
import os

file_paths = {
    'admissions': 'rawcsvs/ADMISSIONS.csv',
    'microbiology_events': 'rawcsvs/MICROBIOLOGYEVENTS.csv',
    'prescriptions': 'rawcsvs/PRESCRIPTIONS.csv',
    'labevents': 'rawcsvs/LABEVENTS.csv',
    'diagnoses_icd': 'rawcsvs/DIAGNOSES_ICD.csv',
    'patients': 'rawcsvs/PATIENTS.csv',
    'transl-labitems': 'rawcsvs/D_LABITEMS.csv',
    'transl-diag': 'rawcsvs/D_ICD_DIAGNOSES.csv'
}

cols_to_use = {
    'admissions': ['subject_id','admittime','ethnicity', 'admission_type'],
    'microbiology_events': ['row_id', 'subject_id',  'chartdate', 'charttime', 'spec_itemid', 'spec_type_desc', 
                            'org_itemid', 'org_name', 'isolate_num', 'ab_itemid', 'ab_name', 'dilution_text', 
                            'dilution_comparison', 'dilution_value', 'interpretation'],
    'prescriptions': ['subject_id', "hadm_id", 'drug', 'enddate'],
    'labevents': ['subject_id', 'itemid', 'valuenum', 'valueuom', 'flag'],
    'diagnoses_icd': ['subject_id', 'icd9_code'], #gotta work on this one, adding all codes to one line in the array
    'transl-labitems': ['itemid', 'label'], # use this in conjunction with labevents to understando wtf if happening
    'patients': ['subject_id', 'gender'],
    'transl-diag': ['icd9_code', 'short_title', 'long_title']
}

files = {}
for file_path in file_paths:
    files[file_path] = pd.read_csv(file_paths[file_path], usecols=cols_to_use[file_path])
    
for file_path, file in files.items(): 
    if  not file_path.startswith('transl'):
        file.dropna(inplace=True, subset=['subject_id'])
        file.drop_duplicates(inplace=True)  
        file_name = os.path.basename(file_path)
        file.to_csv(f'cleanedcsv/{file_name}.csv', index=False)

MANEIRA PARA JUNTAR TODOS OS DADOS


In [4]:
'''
Isso esta feito para essas duas tabelas
o modo nao mudou, precisa mudar as tabelas importadas, as colunas pivoteadas, e o valor de value, dependendo do que 
precisa ser pivoteado
Agradeco Pedro por me dar a idea de pivotear as tabelas
'''
import pandas as pd
mergedata = pd.read_csv('mergedcsv/admin-diag-lab-micro.csv')
diagnoses = pd.read_csv('cleanedcsv/prescriptions.csv')


#inverter a tabela de diagnósticos para ter uma linha por paciente

diaginv = diagnoses.assign(value=1).pivot_table(
    index='subject_id', 
    columns='drug', 
    values='value', 
    fill_value=0
).reset_index()    # Reset the index to make `subject_id` a column

diaginv.fillna(0, inplace=True)

# Now proceed with the merge
mergedata = mergedata.merge(diaginv, on='subject_id', how='left')




TRADUCAO DE CODIGOS PARA TEXTOS


In [5]:
import pandas as pd
#Need to convert the icd9_codes into text
def tryconvertint(possibleint):
    try:
        x = int(possibleint)
        return x
    except ValueError:
        return possibleint 

#first, I will get a representation of all the possible codes and their respective values
unique_diag_codes = pd.read_csv('rawcsvs\D_ICD_DIAGNOSES.csv').filter(['icd9_code', 'short_title']).drop_duplicates()
unique_labitem_codes = pd.read_csv('rawcsvs\D_LABITEMS.csv').filter(['itemid', 'label']).drop_duplicates()

#second, I will iterate through each column name and replace it with the title and/or label
allthedata = pd.read_csv('mergedcsv\\admin-diag-lab-micro-pat-prescrip.csv')

new_columns = []

for column in allthedata.columns:
    colint = tryconvertint(column)
    if colint == 50800:
        x = unique_labitem_codes['itemid'].values
        pass

    # Check if the column is in the 'icd9_code' of unique_diag_codes
    if column in unique_diag_codes['icd9_code'].values:
        # Map it to 'short_title'
        new_columns.append(unique_diag_codes.loc[unique_diag_codes['icd9_code'] == column, 'short_title'].values[0])

    # Check if the column is in the 'itemid' of unique_labitem_codes
    elif colint in unique_labitem_codes['itemid'].values:
        # Map it to 'label'
        new_columns.append(unique_labitem_codes.loc[unique_labitem_codes['itemid'] == colint, 'label'].values[0])

    # If not found, retain the original column name
    else:
        new_columns.append(column)

allthedata.columns = new_columns

allthedata.to_csv('mergedcsv/final_merged_data.csv', index=False)


<>:11: SyntaxWarning: invalid escape sequence '\D'
<>:12: SyntaxWarning: invalid escape sequence '\D'
<>:11: SyntaxWarning: invalid escape sequence '\D'
<>:12: SyntaxWarning: invalid escape sequence '\D'
C:\Users\bitan\AppData\Local\Temp\ipykernel_36608\395378359.py:11: SyntaxWarning: invalid escape sequence '\D'
  unique_diag_codes = pd.read_csv('rawcsvs\D_ICD_DIAGNOSES.csv').filter(['icd9_code', 'short_title']).drop_duplicates()
C:\Users\bitan\AppData\Local\Temp\ipykernel_36608\395378359.py:12: SyntaxWarning: invalid escape sequence '\D'
  unique_labitem_codes = pd.read_csv('rawcsvs\D_LABITEMS.csv').filter(['itemid', 'label']).drop_duplicates()


EXTRACAO DE DADOS IMPORTANTES A BUSCA


In [6]:
import Levenshtein
#supunhetemos que tenhamos uma busca
search = ab_name
#precisamos listar todas as doencas em que o medicamento foi usado
#para facilitar a busca, precisamos calcular o medicamento mais provavel que foi escrito
all_drugs = files['microbiology_events']['ab_name'].dropna().unique()

closest_drug = None	
for drug in all_drugs:
    dist = Levenshtein.distance(search, drug)
    if closest_drug is None or dist < closest_drug[1]:
        closest_drug = (drug, dist)

#agora que temos o medicamento mais provavel, podemos listar as resistencias

NameError: name 'ab_name' is not defined

Coletando os dados de resistencia


In [16]:
import plotly.express as px

#need to sort he diseases
closest_drug = files['microbiology_events']['ab_name'].mode()[0]
diseases = files['microbiology_events'][files['microbiology_events']['ab_name'] == closest_drug[0]]['org_name'].unique()



dis_res_count = {}
for disease in diseases:
    disease_res = files['microbiology_events'][
        (files['microbiology_events']['ab_name'] == closest_drug[0]) & 
        (files['microbiology_events']['org_name'] == disease)
    ]['interpretation']
    
    # Initialize counts for 'R', 'I', 'S'
    counts = {'R': 0, 'I': 0, 'S': 0}
    
    counts.update(disease_res.value_counts().to_dict())
    
    dis_res_count[disease] = counts





FODASSE O MODELO PREDITIVO


In [30]:
from flask import Flask, render_template, request, jsonify, redirect, url_for, session
import Levenshtein

app = Flask(__name__)
app.secret_key = 'VAI SE FUDER FIGA'

@app.route('/', methods=['GET', 'POST'])
def home():
    if request.method == 'GET':
        return render_template('busca.html')
    elif request.method == 'POST':
        ab_name = request.form.get('ab_name')

        def search(term):
            all_drugs = files['microbiology_events']['ab_name'].dropna().unique()

            closest_drug = None    
            for drug in all_drugs:
                dist = Levenshtein.distance(term, drug)
                if closest_drug is None or dist < closest_drug[1]:
                    closest_drug = (drug, dist)
            return closest_drug
        

        if ab_name:
            droga = search(ab_name)
            session['droga'] = droga[0]  # Armazena apenas o nome da droga na sessão
            return redirect(url_for('resposta'))
        else:
            return jsonify({"error": "Nome do antibiótico não informado"}), 400

# Rota para a página secundaria (resposta)

@app.route('/resposta')

def resposta():
    closest_drug = session.get('droga')  # Recupera a droga da sessão

    # Verifica se o medicamento foi encontrado
    if closest_drug:
        # Chama a função que calcula as contagens de resistência
        resistencias = dis_res_count(closest_drug)
    else:
        resistencias = {}

    # Renderiza o template com os dados de resistência
    return render_template('resposta.html', closest_drug=closest_drug, resistencias=resistencias)

if __name__ == '__main__':app.run() 


def dis_res_count(closest_drug):
    # Filtra as doenças baseadas no medicamento mais próximo
    diseases = files['microbiology_events'][files['microbiology_events']['ab_name'] == closest_drug]['org_name'].unique()
    
    # Dicionário para armazenar a contagem de resistência por doença
    dis_res_count = {}

    # Itera sobre cada doença e conta as interpretações 'R', 'I', 'S'
    for disease in diseases:
        disease_res = files['microbiology_events'][
            (files['microbiology_events']['ab_name'] == closest_drug) & 
            (files['microbiology_events']['org_name'] == disease)
        ]['interpretation']
        
        # Inicializa a contagem de 'R', 'I', 'S'
        counts = {'R': 0, 'I': 0, 'S': 0}
        
        # Atualiza as contagens com base nas interpretações
        counts.update(disease_res.value_counts().to_dict())
        
        # Armazena a contagem da doença atual
        dis_res_count[disease] = counts

    return dis_res_count


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [19/Oct/2024 17:00:00] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2024 17:00:00] "GET /static/estilos.css HTTP/1.1" 304 -
127.0.0.1 - - [19/Oct/2024 17:00:00] "GET /static/busca.js HTTP/1.1" 304 -
127.0.0.1 - - [19/Oct/2024 17:00:02] "POST / HTTP/1.1" 302 -
127.0.0.1 - - [19/Oct/2024 17:00:02] "GET /resposta HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2024 17:00:02] "GET /resposta HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2024 17:00:02] "GET /static/resposta.css HTTP/1.1" 304 -


SUGESTAO


In [10]:
def drug_suggestion(org_name, resistance):
    if resistance == 'R':
        return "Sugestão de antibiótico alternativo"
    else:
        return "Sugestão de antibiótico padrão"

merged_data['sugestao'] = merged_data.apply(lambda x: drug_suggestion(x['org_name'], x['interpretation']), axis=1)


NameError: name 'merged_data' is not defined

FERRAMENTA VALIDAÇÃO


In [ ]:
from sklearn.metrics import accuracy_score

# Testar a ferramenta em um conjunto de dados de validação
resultados_reais = merged_data1['antibiotico_real']
sugestoes = merged_data1['sugestao']

acuracia = accuracy_score(resultados_reais, sugestoes)
print(f"Acurácia da ferramenta: {acuracia}")
